### Проверить текущую конфигурацию Spark (spark-defaults.conf)

In [ ]:
#===========================================================================
# Default spark conf in file:
# cat /usr/local/spark/conf/spark-defaults.conf
#===========================================================================
!cat /opt/spark/conf/spark-defaults.conf

### Создать локальную Spark сессию 

* Spark сессию можно не создавать явно. При выполнении любой ячейки с магической функцией *%%sql* сессия создается автоматически

In [10]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Jupyter") \
.getOrCreate()

25/10/02 13:00:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


##### Остановить Spark сессию (если необходимо)

In [ ]:
spark.stop()

##### Мета информация о доступных каталогах
для того чтобы listCatalogs вырнул информацмю о всех подключенных каталогах, необходимо сначала внутри сессии оратиться к одному объекту каждого каталога

### Работа с каталогом Iceberg

##### Посмотреть список схем в каталоге

In [ ]:
%%sql
show schemas

##### Создать схему

In [2]:
%%sql
CREATE schema if NOT EXISTS dwh

++
||
++
++

##### Посмотреть список таблиц в схеме

In [ ]:
%%sql
show tables from dwh

---
### Cоздать таблицу

In [ ]:
%%sql
drop table dwh.example_data PURGE

In [1]:
%%sql
CREATE TABLE IF NOT EXISTS dwh.example_data (
    id int,
    name string,
    phoneNumber string,
    email string,
    address string,
    userAgent string,
    hexcolor string
  ) 
USING iceberg
TBLPROPERTIES ('write.spark.accept-any-schema'='true')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/02 12:50:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/02 12:50:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/10/02 12:50:42 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to spark-history/local-1759409439695.inprogress. This is unsupported


++
||
++
++

##### Создаем view для данных в csv файле

In [3]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW tmpv_example_data
USING csv
OPTIONS (
    path 'example_data.csv',
    header 'true',
    inferSchema 'true',
    quote '"'
    
);

++
||
++
++

##### Вставить данные в таблицу

In [4]:
%%sql
INSERT INTO dwh.example_data
SELECT *
FROM tmpv_example_data

++
||
++
++

##### Прочитать данные таблицы 

In [ ]:
%%sql
SELECT * FROM dwh.example_data

### Создать таблицу с партициями 

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS dwh.example_data_with_partitions (
    id int,
    name string,
    phoneNumber string,
    email string,
    address string,
    userAgent string,
    hexcolor string
  ) 
USING iceberg
TBLPROPERTIES ('write.spark.accept-any-schema'='true')
PARTITIONED BY (hexcolor)

In [ ]:
%%sql
INSERT INTO dwh.example_data_with_partitions
SELECT *
FROM tmpv_example_data

---

### Посмотреть структуру файлов на s3

In [ ]:
!aws s3 ls s3://warehouse/iceberg_rest/dwh/example_data/ --recursive --endpoint-url=http://minio-s3:9080

In [ ]:
!aws s3 ls s3://warehouse/iceberg_rest/dwh/example_data_with_partitions/ --recursive --endpoint-url=http://minio-s3:9080

---

### Просмотр мето-информации о таблице
https://iceberg.apache.org/docs/nightly/spark-queries/#inspecting-tables

In [ ]:
%%sql
DESCRIBE FORMATTED dwh.example_data;

In [6]:
%%sql 
-- snapshots
SELECT * FROM dwh.example_data.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-10-02 12:54:17.493000,4962268757432518498,None,append,s3://warehouse/iceberg_rest/dwh/example_data/metadata/snap-4962268757432518498-1-981e0c5f-90f6-4c77-affd-aa8ecef774b6.avro,"{'engine-version': '3.4.0', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1759409439695', 'added-records': '4', 'total-records': '4', 'spark.app.id': 'local-1759409439695', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '3286', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.9.1 (commit f40208ae6fb2f33e578c2637d3dea1db18739f31)', 'total-files-size': '3286', 'total-data-files': '1'}"
2025-10-02 12:55:51.099000,5795304953496581163,4962268757432518498,append,s3://warehouse/iceberg_rest/dwh/example_data/metadata/snap-5795304953496581163-1-4f192237-7477-4196-af2e-b99ee5862af9.avro,"{'engine-version': '3.4.0', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1759409439695', 'added-records': '1', 'total-records': '5', 'spark.app.id': 'local-1759409439695', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '650', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.9.1 (commit f40208ae6fb2f33e578c2637d3dea1db18739f31)', 'total-files-size': '3936', 'total-data-files': '2'}"


In [ ]:
%%sql
-- history
SELECT * FROM dwh.example_data.history

In [7]:
%%sql 
-- files
SELECT * FROM dwh.example_data.files

content,file_path,file_format,spec_id,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,first_row_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/iceberg_rest/dwh/example_data/data/00000-3-4e3e8a48-12f8-405c-901a-54c66b22fc22-0-00001.parquet,PARQUET,0,1,650,"{1: 39, 2: 42}","{1: 1, 2: 1}","{1: 0, 2: 0}",{},"{1: bytearray(b'c\x00\x00\x00'), 2: bytearray(b'Bob')}","{1: bytearray(b'c\x00\x00\x00'), 2: bytearray(b'Bob')}",None,[4],None,0,None,None,None,None,"Row(address=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), email=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), hexcolor=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), id=Row(column_size=39, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=99, upper_bound=99), name=Row(column_size=42, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='Bob', upper_bound='Bob'), phoneNumber=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), userAgent=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None))"
0,s3://warehouse/iceberg_rest/dwh/example_data/data/00000-2-e6d299af-2b4b-4316-b99e-f56a8fdea30c-0-00001.parquet,PARQUET,0,4,3286,"{1: 57, 2: 117, 3: 102, 4: 133, 5: 228, 6: 267, 7: 80}","{1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4}","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}",{},"{1: bytearray(b'\x01\x00\x00\x00'), 2: bytearray(b'Bo Breitenberg'), 3: bytearray(b'(602) 248-6549'), 4: bytearray(b'brooklyn96@berge'), 5: bytearray(b'""61975 Hammes Tu'), 6: bytearray(b'Mozilla/5.0 (Mac'), 7: bytearray(b'#2ab5bf')}","{1: bytearray(b'\x04\x00\x00\x00'), 2: bytearray(b'Mr. Terry Halvos'), 3: bytearray(b'401-631-4492'), 4: bytearray(b'pearlie80@paucel'), 5: bytearray(b'65588 Lisandro I'), 6: bytearray(b'Mozilla/5.0 (con'), 7: bytearray(b'#b6002b')}",None,[4],None,0,None,None,None,None,"Row(address=Row(column_size=228, value_count=4, null_value_count=0, nan_value_count=None, lower_bound='""61975 Hammes Tu', upper_bound='65588 Lisandro I'), email=Row(column_size=133, value_count=4, null_value_count=0, nan_value_count=None, lower_bound='brooklyn96@berge', upper_bound='pearlie80@paucel'), hexcolor=Row(column_size=80, value_count=4, null_value_count=0, nan_value_count=None, lower_bound='#2ab5bf', upper_bound='#b6002b'), id=Row(column_size=57, value_count=4, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=4), name=Row(column_size=117, value_count=4, null_value_count=0, nan_value_count=None, lower_bound='Bo Breitenberg', upper_bound='Mr. Terry Halvos'), phoneNumber=Row(column_size=102, value_count=4, null_value_count=0, nan_value_count=None, lower_bound='(602) 248-6549', upper_bound='401-631-4492'), userAgent=Row(column_size=267, value_count=4, null_value_count=0, nan_value_count=None, lower_bound='Mozilla/5.0 (Mac', upper_bound='Mozilla/5.0 (con'))"


In [8]:
%%sql 
-- metadata_log_entries
SELECT * FROM dwh.example_data.metadata_log_entries

timestamp,file,latest_snapshot_id,latest_schema_id,latest_sequence_number
2025-10-02 12:50:50.116000,s3://warehouse/iceberg_rest/dwh/example_data/metadata/00000-fa91050b-aa12-42a7-b1b4-f82ad9abe489.metadata.json,None,None,None
2025-10-02 12:54:17.493000,s3://warehouse/iceberg_rest/dwh/example_data/metadata/00001-b1622f17-6676-4b23-9a02-d777eba42338.metadata.json,4962268757432518498,0,1
2025-10-02 12:55:51.099000,s3://warehouse/iceberg_rest/dwh/example_data/metadata/00002-0f05fefa-9205-4722-a241-ae85eb7089fc.metadata.json,5795304953496581163,0,2


In [ ]:
%%sql 
-- entries
SELECT * FROM dwh.example_data.entries

In [ ]:
%%sql 
-- manifests
SELECT * FROM dwh.example_data.manifests

In [ ]:
%%sql 
-- partitions
SELECT * FROM dwh.example_data.partitions

In [ ]:
%%sql 
-- all_data_files
SELECT * FROM dwh.example_data.all_data_files

In [ ]:
%%sql 
-- all_delete_files
SELECT * FROM dwh.example_data.all_delete_files

In [ ]:
%%sql 
-- all_entries
SELECT * FROM dwh.example_data.all_entries

---

### Изменение данных

##### Добавление строк

In [5]:
%%sql
INSERT INTO dwh.example_data (id, name)
Values (99, 'Bob')

++
||
++
++

##### Опеация Update

In [ ]:
%%sql
update dwh.example_data 
   set name = 'Jack' 
where id = 2

##### Операция Delete

In [ ]:
%%sql
delete from dwh.example_data 
where id = 5

---

### Roll back
https://iceberg.apache.org/docs/nightly/spark-procedures/#snapshot-management

In [ ]:
%%sql 
-- snapshots
SELECT * FROM dwh.example_data.snapshots

In [ ]:
%%sql
CALL system.rollback_to_snapshot('dwh.example_data', 4332729665917104153) --change snapshot_id

In [ ]:
%%sql
select * from dwh.example_data

### Оптимизация файлов и очистка старых файлов
https://iceberg.apache.org/docs/nightly/spark-procedures/#metadata-management

##### Оптимизируем файлы

In [ ]:
%%sql
CALL system.rewrite_data_files(
    table => 'dwh.example_data',
    options => MAP(
        'target-file-size-bytes', '134217728',  -- 128MB на партицию
        'min-input-files', '1'
    )
)

##### Удаляем старые snapshot'ы

In [ ]:
%%sql
CALL system.expire_snapshots(
    table => 'dwh.example_data',
    older_than => TIMESTAMP '2025-10-03 00:00:00.000'
)

##### Очищаем orphan файлы

In [ ]:
%%sql
CALL system.remove_orphan_files(
    table => 'dwh.example_data'
)

### Что хранит файл манифеста

In [11]:
spark.read.text('s3a://warehouse/iceberg_rest/dwh/example_data/metadata/00001-b1622f17-6676-4b23-9a02-d777eba42338.metadata.json').show(1000,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                              |
+---------------------------------------------------------------------------------------------------------------------------------------------------+
|{                                                                                                                                                  |
|  "format-version" : 2,                                                                                                                            |
|  "table-uuid" : "9ee05807-6f1c-43f8-9298-3d1370a74b63",                                                                                           |
|  "location" : "s3://warehouse/iceberg_rest/dwh/example_data",                                     

---

### Демонстрация изменения данных на уровне файловой системы

In [12]:
%%sql
drop table if exists dwh.example_table purge

++
||
++
++

In [13]:
%%sql
create table dwh.example_table(
 id int,
 name string
    
) 

++
||
++
++

In [20]:
!aws s3 ls s3://warehouse/iceberg_rest/dwh/example_table/ --recursive --endpoint-url=http://minio-s3:9080

2025-10-02 13:02:16        669 iceberg_rest/dwh/example_table/data/00000-11-04584a3c-17a7-4193-b81e-b3a4c99b69fc-0-00001.parquet
2025-10-02 13:02:40        707 iceberg_rest/dwh/example_table/data/00000-14-636edef9-3c23-4f49-9e7d-11ecd916932c-0-00001.parquet
2025-10-02 13:02:55        664 iceberg_rest/dwh/example_table/data/00000-16-65cd65b4-ff42-4c13-904e-01ac12ea6767-0-00001.parquet
2025-10-02 13:01:56       1031 iceberg_rest/dwh/example_table/metadata/00000-01fcbeb9-2ca1-460e-951f-847626d3504d.metadata.json
2025-10-02 13:02:16       2321 iceberg_rest/dwh/example_table/metadata/00001-4f936a02-00d5-490e-bf12-9153a6a72510.metadata.json
2025-10-02 13:02:40       3659 iceberg_rest/dwh/example_table/metadata/00002-89b66c92-6106-43be-b06e-d3b6d6cf3cb7.metadata.json
2025-10-02 13:02:55       4889 iceberg_rest/dwh/example_table/metadata/00003-cfbd2c97-8a31-4121-bccb-7d5e882b5dbf.metadata.json
2025-10-02 13:02:55       7035 iceberg_rest/dwh/example_table/metadata/26f6399a-ae04-4824-8207-d94b18

In [15]:
%%sql
insert into dwh.example_table 
select /*+ REPARTITION(1) */  * 
  from values 
    (1, 'row 1'), 
    (2, 'row 2'),
    (3, 'row 3') 

++
||
++
++

In [19]:
%%sql
insert into dwh.example_table 
select /*+ REPARTITION(1) */  * 
  from values 
    (4, 'row 4') 

++
||
++
++

In [17]:
%%sql
update dwh.example_table 
  set name = 'row 2 after update'
where id = 2

++
||
++
++

In [ ]:
%%sql
select * from dwh.example_table 

In [21]:
%%sql
select * from parquet.`s3a://warehouse/iceberg_rest/dwh/example_table/data/00000-11-04584a3c-17a7-4193-b81e-b3a4c99b69fc-0-00001.parquet` 

id,name
1,row 1
2,row 2
3,row 3


In [ ]:
%%sql
select * from parquet.`s3a://warehouse/iceberg_rest/dwh/example_table/data/00000-2997-1f91a811-2ec4-49c9-8176-12852a14f830-0-00001.parquet` 

In [ ]:
%%sql
select * from parquet.`s3a://warehouse/iceberg_rest/dwh/example_table/data/00000-3000-a990db74-939f-4770-8b05-cc479d68134b-0-00001.parquet` 

In [ ]:
%%sql
delete from dwh.example_table 
where id = 3

In [ ]:
%%sql
select * from parquet.`s3a://warehouse/iceberg_rest/dwh/example_table/data/00000-3009-76d45368-4004-4e2a-98b8-cac3340b6b3f-0-00001.parquet`